In [10]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [3]:
iris = load_iris()
X, y = iris.data, iris.target


x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### SVM Multi-Class Support

```
How many classes?
│
├─ 2 classes → Use binary classifier directly
│
├─ 3-10 classes → Consider OvO (often more accurate)
│  │
│  └─ Need speed? → Use OvR
│
└─ > 10 classes → Use OvR (OvO becomes too expensive)
   │
   └─ Small dataset? → Still consider OvO
```

#### One-vs-Rest (OvR) 

In [ ]:
# OvR with SVM (SVM is binary by default)

ovr_svm = OneVsRestClassifier(SVC(probability=True, random_state=42))
ovr_svm.fit(x_train, y_train)

y_pred_ovr = ovr_svm.predict(x_test)

print("One-vs-Rest with SVM:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_ovr):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_ovr, target_names=iris.target_names))

One-vs-Rest with SVM:
Accuracy: 1.000

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



#### One-vs-One (OvO)

In [ ]:
# OvO with SVM (For n classes, train n×(n-1)/2 classifiers)

ovo_svm = OneVsOneClassifier(SVC(random_state=42))
ovo_svm.fit(x_train, y_train)

y_pred_ovo = ovo_svm.predict(x_test)

print("One-vs-Rest with SVM:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_ovo):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_ovo, target_names=iris.target_names))

print(f"\nNumber of binary classifiers: {len(ovo_svm.estimators_)}")

One-vs-Rest with SVM:
Accuracy: 1.000

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


Number of binary classifiers: 3


**OvR vs OvO**

In [11]:
def compare_strategies(x, y, dataset_name):
    print(f"\n{dataset_name} Dataset ({len(np.unique(y))} classes):")
    print("=" * 60)
    
    # OvR
    ovr = OneVsRestClassifier(SVC(random_state=42))
    ovr_scores = cross_val_score(ovr, x, y, cv=5, scoring='accuracy')
    
    # OvO
    ovo = OneVsOneClassifier(SVC(random_state=42))
    ovo_scores = cross_val_score(ovo, x, y, cv=5, scoring='accuracy')
    
    print(f"One-vs-Rest:  {ovr_scores.mean():.3f} (+/- {ovr_scores.std():.3f})")
    print(f"One-vs-One:   {ovo_scores.mean():.3f} (+/- {ovo_scores.std():.3f})")
    
    n_classes = len(np.unique(y))
    print(f"\nNumber of classifiers:")
    print(f"  OvR: {n_classes}")
    print(f"  OvO: {n_classes * (n_classes - 1) // 2}")
    


iris = load_iris()
wine = load_wine()

compare_strategies(iris.data, iris.target, 'Iris')
compare_strategies(wine.data, wine.target, 'Wine')   


Iris Dataset (3 classes):
One-vs-Rest:  0.953 (+/- 0.034)
One-vs-One:   0.973 (+/- 0.013)

Number of classifiers:
  OvR: 3
  OvO: 3

Wine Dataset (3 classes):
One-vs-Rest:  0.686 (+/- 0.034)
One-vs-One:   0.692 (+/- 0.043)

Number of classifiers:
  OvR: 3
  OvO: 3


### Native Multi-Class Support

In [12]:
# These work directly with multi-class problems
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# No need for OvR or OvO wrappers
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB()
}

iris = load_iris()
X, y = iris.data, iris.target

In [13]:
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f"{name:20s}: {scores.mean():.3f} (+/- {scores.std():.3f})")

Decision Tree       : 0.953 (+/- 0.034)
Random Forest       : 0.967 (+/- 0.021)
KNN                 : 0.973 (+/- 0.025)
Naive Bayes         : 0.953 (+/- 0.027)
